<a href="https://colab.research.google.com/github/SarthakNarayan/DL-and-ML/blob/master/googlecolab/Implementing_Resnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Colors section

In [0]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

# print (color.GREEN + 'Hello World !' )
# print("happens if u dont end it")
# print(color.END)
# print("will not happen now")

###Imports Section

In [0]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torchvision import models

##ResNet 50,101,152 architecture
https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

![alt text](https://www.codeproject.com/KB/AI/1248963/resnet.png)<br/>
You can see the layer count on the top right hand corner.<br/>
They use bottleneck blocks whereas as ResNet 18 and 34 uses basic blocks<br/>
You can see number of blocks information below
![alt text](https://cdn-images-1.medium.com/max/1600/1*aq0q7gCvuNUqnMHh4cpnIw.png)

###Implementing Basic Block


![alt text](https://raw.githubusercontent.com/torch/torch.github.io/master/blog/_posts/images/resnets_1.png)

Resnet Image<br/>
Here dotted skip connections imply change of dimension like from 64 to 128, from 128 to 256 and so on.
![alt text](https://storage.googleapis.com/kaggle-datasets-images/6885/9959/d9e74a548a8cdca167b38520ac8bf405/data-original.png?t=2017-12-12-23-54-44)

In [0]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self , input_channels , output_channels , stride = 1 , dim_change = None):
        super(BasicBlock , self).__init__()
        # First stride may differ. It can be 2 but then always 1
        self.conv1 = nn.Conv2d(input_channels , output_channels , stride = stride , kernel_size = 3 , padding = 1)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels , output_channels , stride = 1 , kernel_size = 3 , padding = 1)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.dim_change = dim_change
        
    def forward(self , x):
        residual_connection = x
        output = F.relu(self.bn1(self.conv1(x)))
        # No relu here since we have to add
        output = self.bn2(self.conv2(output))
        
        #But first we will check for dimension change
        if dim_change is not None:
            residual_connection = self.dim_change(residual_connection)
            
        output = residual_connection + output
        output = F.relu(output)
        return output
    
# basic_block = BasicBlock(input_channels = 3, output_channels = 64, stride = 2)
# print(basic_block)

###Implementing Bottle neck block

In [0]:
class BottleNeck(nn.Module):
    # expansion will be used later when we define the full network
    expansion = 4
    def __init__(self , input_channels , output_channels , stride = 1 , dim_change = None):
        super(BottleNeck , self).__init__()
        self.conv1 = nn.Conv2d(input_channels , output_channels , stride = 1 , padding = 1 , kernel_size = 1)
        self.bn1 = nn.Conv2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels , output_channels , stride = stride , padding = 1 , kernel_size = 3)
        self.bn2 = nn.Conv2d(output_channels)
        self.conv3 = nn.Conv2d(output_channels , output_channels*4 , stride = 1 , padding = 1 , kernel_size = 3)
        self.bn3 = nn.Conv2d(output_channels*4)
        self.dim_change = dim_change
        
    def forward(self , x):
        residual_connection = x
        output = F.relu(self.bn1(self.conv1(output)))
        output = F.relu(self.bn2(self.conv2(output)))
        output = self.bn3(self.conv3(output))
        
        if dim_change is not None:
            residual_connection = self.dim_change(residual_connection)
            
        output = output + residual_connection
        output = F.relu(output)
        return output
    

###Assembling the network

In [0]:
class ResNets(nn.Module):
    def __init__(self , block , num_layers , classes = 10):
        '''
        The block can be bottleneck block or basic block and num_layers is the
        number of layers required for each block. num_layers will be a list
        '''
        super(ResNets , self).__init__()
        self.input_size = 64
        # First layer in all is 7*7 convolution with stride 2
        self.conv1 = nn.Conv2d(3, 64 , stride = 2 , kernel_size = 7)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # These 4 segments form the starting part. Then we can start our layers
        
        # here _layer is a private function defined later in this block
        # after the first one stride for all others are 2
        self.layer1 = self._layer(block , 64 , num_layers[0] , stride = 1)
        self.layer2 = self._layer(block , 128 , num_layers[1] , stride = 2)
        self.layer3 = self._layer(block , 256 , num_layers[2] , stride = 2)
        self.layer4 = self._layer(block , 512 , num_layers[3] , stride = 2)
        
        # then the end part
        self.avgpool = nn.AvgPool2d(kernel_size = 4 , stride = 1)
        self.fc = nn.Linear(512*block.expansion , classes)
        
    def _layer(self , block , input_size ,num_layers , stride=1):
        dim_change = None
        # we have to find the condition for change of dimensions
        # basically saying that change for dimensions 128,256,512
        # input size gets updated later
        if stride!=1 or input_size!=self.input_size*block.expansion:
            dim_change = nn.Sequential(nn.Conv2d(self.input_size , input_size*block.expansion , kernel_size = 1 , stride = stride),
                                       nn.BatchNorm2d(input_size*block.expansion))
            
        net_layers = []
        # appending the layers
        net_layers.append(block(self.input_size , input_size , stride , dim_change))
        self.input_size = input_size*block.expansion
        for i in range(1,num_layers):
            net_layers.append(block(self.input_size , input_size))

        return nn.Sequential(*net_layers)
        
    def forward(self , x):
        # The beginning 
        x = F.relu(self.maxpool(self.bn1(self.conv1(x))))
        
        # The layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        # first element is always the batch size 
        x = x.reshape(x.size(0) , -1)
        x = self.fc(x)
        return x      
        

###Creating Resnets

In [31]:
def Creating_Resnets():
    # Resnet 18
    resnet18 = ResNets(BasicBlock , [2,2,2,2] , 10)
    print(resnet18)
    
    # for comparing 
    builtin_resnet18 = models.resnet18()
    print(color.RED + "\n Built in Resnet for comparison \n" + color.END)
    print(builtin_resnet18)
    dummy_input = torch.randn(1, 3, 28, 28)
    torch.onnx.export(builtin_resnet18, dummy_input, "resnet18.onnx")
    
    # for other resnets like 50
#     resnet50 = ResNets(BottleNeck , [3,4,6,3] , 10)
#     print(resnet50)
    
Creating_Resnets()

ResNets(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

##Viewing the model

In [3]:
!pip install onnx


     |████████████████████████████████| 7.1MB 5.0MB/s 


NameError: ignored